In [1]:
import pandas as pd
import os
import cv2
import numpy as np
import json

In [3]:
#Test Code| Version 2#
Year=
Showa=
#Read Motherdata
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\"+Year+"\\DataFrame.json"
with open(path, 'r') as j:
     dta = json.loads(j.read())

In [16]:
#Test code| Version 2#
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Office_Level\\"+Year+"\\"
os.chdir(path)
df = pd.read_csv(r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Processed_Data/Index/S'+Showa+'.csv')
df=df.drop(df.columns[0], axis=1)

PositionList=['Manager','Leader','Admin','Outsource','Engineer1','Engineer2']

In [17]:
# Function for returning x coordinate of Position name
def Detect_Position(Json,Position):
    NewList=Json['fields']
    Dict=list()
    for d in NewList:
        try:
            newDict={}
            newDict['inferText']=d['inferText']
            newDict['boundingPoly']=d['boundingPoly']
            Dict.append(newDict)
        except KeyError:
            continue
            
    ### Get Coordinates ###
    if Position=='Manager':        
        res = [d
           for d in Dict 
           if ('主事' in d['inferText']) or ('主' in d['inferText']) or ('事' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            return(Edge)
        else:
            return([])
    
    if Position=='Leader':        
        res = [d
           for d in Dict 
           if ('課長' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            return(Edge)
        else:
            return([])
        
    if Position=='Engineer1':        
        res = [d
           for d in Dict 
           if ('技師' in d['inferText']) or ('師' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            return(Edge)
        else:
            return([])
    
    if Position=='Engineer2':        
        res = [d
           for d in Dict 
           if ('技手' in d['inferText']) or ('手' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            return(Edge)
        else:
            return([])
        
    if Position=='Admin':        
        res = [d
           for d in Dict 
           if ('事務員' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            return(Edge)
        else:
            return([])
        
    if Position=='Outsource':
        res = [d
           for d in Dict 
           if '嘱託員' in d['inferText']]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            return(Edge)
        else:
            return([])
        
    if Position=='Clerk':
        res = [d
           for d in Dict 
           if d['inferText']=='書記']
        if len(res)!=0:
            res = res[0]['boundingPoly']
            Edge = max(res, key = lambda i : i[0])[0]
            return(Edge)
        else:
            return([])
        
    if Position=='Worker':
        res = [d
           for d in Dict 
           if d['inferText']=='雇']
        if len(res)!=0:
            res = res[0]['boundingPoly']
            Edge = max(res, key = lambda i : i[0])[0]
            return(Edge)
        else:
            return([])
        
### CLOVA FUNCTION ###
import requests
import uuid
import time
import json
import cv2
import base64

api_url = 'https://deelieyxuc.apigw.ntruss.com/custom/v1/1972/ebd01bcbf693d069817622e9839e20914143c7d0d8953eddee40e8b0af96c95b/general'
secret_key = 'S1NmVXpYZlJ0cGJ0ZEFRZXVlbkRkaHFReE9FcHNTQ0U='

def Clova(Dept,Office,ImagePage):
    file_path=path+Dept+'\\'+Office+'\\'+"Image"+str(ImagePage)+".png"
    stream = open(file_path, "rb")
    bytes = bytearray(stream.read())
    numpyarray = np.asarray(bytes, dtype=np.uint8)
    img = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)
    temp_path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Temp\\"
    HH,WW=img.shape[:2]

    Cropped=img[0:150,0:WW]
    cv2.imshow('img',Cropped)
    cv2.waitKey(0)
    cv2.imwrite(temp_path+"Temp.jpg",Cropped)
    with open(temp_path+"Temp.jpg",'rb') as f:
        file_data = f.read()
    
    request_json = {
            'images': [
                {
                    'format': 'jpg',
                    'name': 'demo',
                    'data':base64.b64encode(file_data).decode()}],
            'requestId': str(uuid.uuid4()),
            'version': 'V2',
            'timestamp': int(round(time.time() * 1000)),
            'lang':'ja'
            }
    payload = json.dumps(request_json).encode("UTF-8")
    headers = {'X-OCR-SECRET': secret_key,
              'Content-Type': 'application/json'}
    response = requests.request("POST", api_url, headers=headers, data = payload)
    Json=json.loads(response.text)['images'][0]
    
    return Json    


class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)


In [4]:
#Test code| Version 2#
for n in range(1,len(df)+1):
    PositionListNow=PositionList.copy()
    #Show Working office#
    #Extract key info of office
    if n==len(df):
        print('FINISHED!')
    Row  = df.iloc[n]
    NxRow= df.iloc[n+1]


    ###Collect Location information###
    Page=int(Row["Page"])
    NxPage=int(NxRow["Page"])

    Dept=Row["Dept"]
    NxDept=NxRow["Dept"]

    Office=Row["Office"]
    NxOffice=NxRow["Office"]

    print(Dept,Office)

    #Convert to json via CLOVA

    print(dta[str(Year)][Dept][Office]['Page_Range'])
    if dta[str(Year)][Dept][Office]['Page_Range']=='NA':
        continue
    CordDict={}
    for l in range(1,len(dta[str(Year)][Dept][Office]['Page_Range'])+1):
        try:
            Json=Clova(Dept,Office,l)
        except FileNotFoundError:
            print('Error')
            break

        if 'fields' not in Json:
            print('CLOVA did not work for '+str(l))
            continue

        #Find Starting X coordinates of Positions.

        CordList=list(map(Detect_Position,[Json]*len(PositionListNow),PositionListNow))
        Positions= {PositionListNow[i]:CordList[i] for i in range(len(PositionListNow))}
        Positions = {k:v for k,v in Positions.items() if v}    
        print(Positions)
        PosiList = list(Positions.keys())
        CordList = list(Positions.values())

        CordDictNow= {PosiList[i]: {'XLocation':CordList[i],'Page':l} for i in range(len(PosiList))}
        CordDictNow = {k:v for k,v in CordDictNow.items() if v}
        CordDict.update(CordDictNow)

        for Posi in list(CordDictNow.keys()):
            XLoc=CordDictNow[Posi]['XLocation']
            print(XLoc)
            file_path=path+"\\"+Dept+"\\"+Office+"\\"+"Image"+str(l)+".png"
            print(file_path)
            stream = open(file_path, "rb")
            bytes = bytearray(stream.read())
            numpyarray = np.asarray(bytes, dtype=np.uint8)
            img = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)

            try:                
                HH=img.shape[:2][0]
                img = cv2.line(img, (XLoc,HH), (XLoc,0), color = (0, 0, 0), thickness=3)
                cv2.imshow(Posi,img)
                cv2.waitKey(0)
            except:
                print('Could not find'+path+"Image"+str(Page)+".png")
                continue
        for x in list(CordDictNow.keys()):
            PositionListNow.remove(x)
    dta[Year][Dept][Office]['Position']=CordDict
    ##Add Ending information
    Frame=pd.DataFrame.from_dict(dta[Year][Dept][Office]['Position'],orient='index')
    if len(Frame)==0:
        continue
    Frame=Frame.sort_values(by = ['Page', 'XLocation'], ascending = [True, False])
    Frame["EndLocation"]=list(Frame['XLocation'].shift(-1))[:-1]+list([dta[Year][Dept][Office]["Office_X2"]])
    EndPage=float(len(dta[str(Year)][Dept][Office]['Page_Range']))
    Frame["EndPage"]=[int(d) for d in [float(i) for i in list(Frame['Page'].shift(periods=-1)[:-1])]+list(str(EndPage)[0])]

    Frame=Frame.to_dict('index')
    dta[Year][Dept][Office]['Position']=Frame

(Office      文書課
 Dept      Admin
 Year          9
 Page          3
 Name: 2, dtype: object,
 Office    庶務課
 Dept      監査局
 Year        9
 Page        5
 Name: 3, dtype: object)

In [10]:
json_object = json.dumps(dta, indent=4,
                        cls=NpEncoder)
save_path='C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\'+str(Year)+'\\'
with open(save_path+"DataFrame.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
df2=pd.DataFrame(columns=['Dept','Office','Year','Position'])
for n in range(0,len(df)):
    Row=df.iloc[n]
    Office=Row['Office']
    Dept=Row['Dept']
    try:
        PosiList=list(dta[Year][Dept][Office]['Position'].keys())
        for Posi in PosiList:
            NewRow={'Office':Office,'Dept':Dept,'Year':Year,'Position':Posi}
            df2 = df2.append(NewRow, ignore_index = True)
    except:
        continue
df2.to_csv(r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Processed_Data/Index/S'+Showa+'_Final.csv')